### Comprehension Burden for Sequencing Documents

In [344]:
import requests
from collections import Counter
import random as randomlib

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import networkx as nx



pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [62]:
'''
Knowledge Graph
'''
kg_path = "../graph_query/graphs/knowledge_graph.gpickle"
kg = nx.read_gpickle(kg_path)
kg_labels = [str(x) for x in list(kg.nodes())[1:]]
n_labels = len(kg_labels)

In [284]:
def get_queries_based_on_node(node_label):
    node = kg.node[node_label]
    
    node_type = node["NodeType"]

    if(node_type == "TopicNode" or node_type == "ConceptNode"):
        return list(kg.neighbors(node_label))
    elif(node_type == "SubConceptNode"):
        return [node_label]
    else:
        pass


    '''
    Returns a list of queries depending on the type of the
    node closest to the query.
    args - query(str)
    returns [] of str
    '''
def query_formulator(query, label):
    queries = []
    children_neighbours = get_queries_based_on_node(label)
    queries = [label]
    for child in children_neighbours:
        queries.append(child)
    return list(set(queries))


In [362]:
'''
Get content from a given set of URLs.
'''
def get_content():
    f = open('user_study_dynamic_programming_engage.txt', 'r')
    l = f.readlines()
    docs = {}
    index = {}
    
    counter = 0
    for url in l:
        docs[url] = requests.get(url).content
        index[url] = counter
        counter += 1
    return docs, index

In [363]:
'''
Term Frequency Array for a particular document.
'''
def get_tfd(content):
    word_count_dict = Counter(w for w in kg_labels 
                              if w.lower() in content.lower())
    common = word_count_dict.most_common()
    
    frequency_arr = [0]*len(kg_labels)
    
    for common_word in common:
        common_word_index = kg_labels.index(common_word[0])
        frequency_arr[common_word_index] = common_word[1]
    return frequency_arr

In [364]:
content, index = get_content()

In [365]:
'''
Building word_data a document (rows) by term frequency (columns) matrix.
'''
tfd_data = {}
for url, cont in content.items():
    tfd_data[url] = get_tfd(cont)

tfd_arr = []
for key in index.keys():
    tfd_arr.append(key.replace("\n", ""))

word_data = {'TFD':tfd_arr}

for label in kg_labels:
    word_data[label] = [None]*len(index)

for url, words_in_doc in tfd_data.items():
    url_index = index[url]
    for i in range(0, n_labels, 1):
        word = kg_labels[i]
        word_data[word][url_index] = words_in_doc[i]

In [366]:
'''
(DTF)^T(DTF) = Coocurence Matrix
'''
document_term_frequency = pd.DataFrame(word_data).set_index('TFD')
dtf_asint = document_term_frequency.astype(int)
coocc = dtf_asint.T.dot(dtf_asint)

### Calculating Relationship Score: S(i, j)

In [367]:
def get_relationship_between_concepts(concept_1, concept_2):
    concept_1_index= document_term_frequency.columns.get_loc(concept_1)
    concept_2_index= document_term_frequency.columns.get_loc(concept_2)
    
    return coocc.iloc[concept_1_index, concept_2_index]

### Significance of a concept in a document: \lambda(c, i)

In [368]:
def get_significance_score(concept, document):
    if(document == None): return 0
    concept_index = document_term_frequency.columns.get_loc(concept)
    freq = dtf_asint.iloc[index[document]][concept_index]
    coocc_row = coocc.iloc[concept_index,:] 
    r = np.array(coocc_row)
    if(sum(r) == 0): return 0
    return (freq)*np.count_nonzero(r)

### Key Sections k_c

In [369]:
key_doc = {}

doc_to_key = {}

labs = []
q = ["Dynamic Programming", "Dijkstra's algorithm"]
for each in q:
    labs.extend(query_formulator(each, each))
    
# labs.append("Greedy Algorithms")
# for each_concept in labs:
#     key_max = 0
#     for each_document in content.keys():
#         s = get_significance_score(each_concept, each_document)
#         if(s > key_max and s > 30):
#             key_max = s
#             key_doc[each_concept] = each_document

# for key, val in key_doc.items():
#     doc_to_key[val] = key

for each_document in content.keys():
    rt = []
    rc = []
    for each_concept in labs:
        s = get_significance_score(each_concept, each_document)
        if(s > 0):
            if(kg.node[each_concept]["NodeType"] != "TopicNode" and 
               kg.node[each_concept]["NodeType"] != "SubjectNode" and
              kg.node[each_concept]["NodeType"] != "CourseNode"):
                rc.append((each_concept, s))
            else:
                if(kg.node[each_concept]["NodeType"] == "TopicNode"):
                    rt.append((each_concept, s))
    rt.sort(key=lambda x: x[1])
    rt = rt[::-1]
    
    rc.sort(key=lambda x: x[1])
    rc = rc[::-1]
    if(len(rc)):
        key_doc[each_document] = rc[0][0]
    else:
        if(len(rt)):
            key_doc[each_document] = rt[0][0]

In [370]:
relevant_concepts = set()
for d, k in key_doc.items():
    relevant_concepts.add(k)

### Comprehension Burden

In [371]:
def f_cb(sig_score, key_sig_score, relationship):
    return key_sig_score

def get_cb_document(document, document_key_concept, visited):
    key_sig_score = get_significance_score(document_key_concept, document)
    document_burden = 0.0
    num_of_docs = 0
    
    order = list(relevant_concepts)
    
    for other_concept in order:
        if(other_concept in visited or other_concept==document_key_concept): continue
        sig_score = get_significance_score(other_concept, document)
        relationship = get_relationship_between_concepts(document_key_concept, other_concept)
        if(sig_score > 0): 
            document_burden += f_cb(sig_score, key_sig_score, relationship)
            num_of_docs += 1
    return document_burden

In [372]:
print(relevant_concepts)

set(['Dynamic Programming', 'Rod cutting', 'Longest common subsequence'])


### Sequence Generation

In [373]:
def get_linear(nodes):
    parents = []
    for each in nodes:
        if(kg.nodes[each]["NodeType"] == "TopicNode"):
            parents.append(each)
    
    linear = []
    for p in parents:
        linear.append(p)
        children = kg.neighbors(p)
        for c in children:
            if c in nodes and kg.nodes[c]["NodeType"] == "ConceptNode":
                linear.append(c)
    
    for each in nodes:
        if each not in linear:
            linear.append(each)
    return linear

def get_bfs(nodes):
    
    return []

def get_random(nodes):
    vals = list(nodes)
    r = []
    while(vals):
        s = randomlib.choice(vals)
        r.append(s)
        vals.remove(s)
    return r
    

def get_sequences(nodes):
    
        random = get_random(nodes)
        linear = get_linear(nodes)
        top_down = linear[::-1]
        return random, linear, top_down
    
#     elif(topic == "Memory Management"):
#         random = get_random(nodes)
#         bfs = ['Memory Management','Computer Science', 'Graph Theory', 'Dfs', 'Supervised Learning', 'Pca', 'Minimum and Maximum']
#         linear = ['Computer Science', 'Memory Management', 'Graph Theory', 'Dfs', 'Supervised Learning', 'Pca', 'Minimum and Maximum']
#         top_down = linear[::-1]
#         return random, linear, bfs, top_down
#     elif(topic == "Mixture Models"):
#         random = get_random(nodes)
#         bfs = ['Memory Management','Computer Science', 'Graph Theory', 'Dfs', 'Supervised Learning', 'Pca', 'Minimum and Maximum']
#         linear = ['Computer Science', 'Memory Management', 'Graph Theory', 'Dfs', 'Supervised Learning', 'Pca', 'Minimum and Maximum']
#         random = linear[::-1]
#         top_down = linear[::-1]
#         return random, linear, bfs, top_down
#     elif(topic == "Dynamic Programming"):
#         random = get_random(nodes)
#         bfs = ["Dijkstra's algorithm",'Depth First Search','Dynamic Programming','Analysis of Algorithms',
#                'Linear Algebra','Operating Systems', "Threads"]
#         linear = ['Analysis of Algorithms', 'Dynamic Programming', 'Depth First Search', 
#                   "Dijkstra's algorithm", 'Linear Algebra','Threads','Operating Systems']
#         random = linear[::-1]
#         top_down = linear[::-1]
#         return random, linear, bfs, top_down

In [374]:
def get_cp_score(order, doc_to_key):
    visited = set()
    total = 0
    ordered = {}

    for each in order: ordered[each] = []
    for doc, kc in doc_to_key.items():
        ordered[kc].append(doc)

    for kc in order:
        visited.add(kc)
        docs = ordered[kc]
        for each in docs:
            total += get_cb_document(each, kc, visited)
    print(total)
    

In [375]:
random, linear, bottom_up = get_sequences(relevant_concepts)
get_cp_score(linear, key_doc)
get_cp_score(bottom_up, key_doc)
get_cp_score(random, key_doc)

0.0
669.0
156.0


In [336]:
print(kg.node["Topological Sort"])
for each in kg.neighbors("Depth First Search"): print(each)

{'NodeType': 'ConceptNode'}


In [353]:
## linear with taking the weights of a particular node into consideration